In [12]:
%reload_ext autoreload
%autoreload 2

In [27]:

from molformer.utils import get_argparse_defaults, read_config
from molformer.tokenizer import MolTranBertTokenizer
from molformer.model.attention import TestBert
from molformer.model.args import get_parser as ARGS

from attributedict.collections import AttributeDict
import torch 
import pandas as pd
import glob


def get_bert(config, tokenizer):
    bert_model = (
        TestBert(
            config,
            tokenizer.vocab,
            config.seed_path,
            rotate=config.rotate,
            device=config.device,
        )
        .to(config.device)
        .eval()
    )
    bert_model.load_state_dict(dict(torch.load(config.seed_path, map_location=config.device)["state_dict"]), strict=True )
    return bert_model


In [28]:

### defaults
DEFAULTS = AttributeDict(get_argparse_defaults(ARGS()))
TOKENIZER = MolTranBertTokenizer("../../../finetune/bert_vocab.txt")
DEFAULTS.device = 'cpu'
DEFAULTS.seed_path = '/Users/kevinmaikjablonka/Downloads/Pretrained MoLFormer/checkpoints/N-Step-Checkpoint_0_0.ckpt' # Doing this is kind of ugly. In the medium to long run this should be a real configuration system (e.g. Hydra)
### import|model
# register_attention("linear")
BERT = get_bert(DEFAULTS, TOKENIZER)

    



RuntimeError: Error(s) in loading state_dict for TestBert:
	Missing key(s) in state_dict: "pos_emb". 
	Unexpected key(s) in state_dict: "blocks.layers.0.attention.rotaryemb.inv_freq", "blocks.layers.1.attention.rotaryemb.inv_freq", "blocks.layers.2.attention.rotaryemb.inv_freq", "blocks.layers.3.attention.rotaryemb.inv_freq", "blocks.layers.4.attention.rotaryemb.inv_freq", "blocks.layers.5.attention.rotaryemb.inv_freq", "blocks.layers.6.attention.rotaryemb.inv_freq", "blocks.layers.7.attention.rotaryemb.inv_freq", "blocks.layers.8.attention.rotaryemb.inv_freq", "blocks.layers.9.attention.rotaryemb.inv_freq", "blocks.layers.10.attention.rotaryemb.inv_freq", "blocks.layers.11.attention.rotaryemb.inv_freq". 
	size mismatch for blocks.layers.0.attention.inner_attention.feature_map.omega: copying a param with shape torch.Size([64, 32]) from checkpoint, the shape in current model is torch.Size([96, 32]).
	size mismatch for blocks.layers.1.attention.inner_attention.feature_map.omega: copying 

In [20]:
DEFAULTS.seed_path

AttributeError: 'seed_path'

In [ ]:
def get_full_model(CHECKPOINT_PATH : str, VOCAB_PATH : str, CONFIG_PATH):
    DEFAULTS = AttributeDict(get_argparse_defaults(ARGS()))
    CONFIG_FILE_PATH = '/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/hparams.yaml'
    CONFIG = read_config(CONFIG_FILE_PATH)
    
    CONFIG.seed_path = CHECKPOINT_PATH
    TOKENIZER = MolTranBertTokenizer(VOCAB_PATH)
    TOKENIZER.name_or_path = VOCAB_PATH
    BERT = get_bert(CONFIG, TOKENIZER)
    return BERT


def agg_attention(smiles : str, nr_of_layers, CONFIG_PATH, VOCAB_PATH):
    for layer in range(nr_of_layers):
        if layer == 0:
            
            attention_tensor = calc_attention_tensor(smiles, layer, CONFIG_PATH, VOCAB_PATH)
        
        else:
            attention_tensor += calc_attention_tensor(smiles, layer, CONFIG_PATH, VOCAB_PATH)
    
    return attention_tensor/nr_of_layers


def r2(x, y):
    x = x.flatten()
    y = y.flatten()
    LR = LinearRegression()
    LR.fit(np.array(x).reshape(-1, 1), y)
    y_pred = LR.predict(np.array(x).reshape(-1, 1))

    return r2_score(y_pred, y)



In [ ]:
DATA_PATH = '/home/mi52qep/proj/molformer_refactor/src/molformer/data/bace/test.csv'
BACE_TEST = pd.read_csv(DATA_PATH).smiles
CHECKPOINT_PATHS = glob.glob("/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/checkpoints/*.ckpt")
VOCAB_PATH = '/home/mi52qep/proj/molformer_refactor/src/molformer/data/bert_vocab.txt'


CONFIG_FILE_PATH = '/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/hparams.yaml'
CONFIG = read_config(CONFIG_FILE_PATH)
CONFIG.seed_path = CHECKPOINT_PATH
CONFIG.canonical = False
CONFIG.mask = False

TEST_SMILES = BACE_TEST[0]

for CHECKPOINT_PATH in CHECKPOINT_PATHS:
    model = get_full_model(CHECKPOINT_PATH, VOCAB_PATH, CONFIG_FILE_PATH)
    attention = agg_attention(TEST_SMILES, 12, CONFIG, VOCAB_PATH)
    distance_matrix = distance_matrix(TEST_SMILES)
    r2 = r2(attention, distance_matrix)
    %%capture
    print(r2)
